<a href="https://colab.research.google.com/github/RicottaChz/Pytorch_Youtube/blob/main/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# Data Load and Preprocessing
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

train_size = int(0.8 * len(data_set))
val_size = len(data_set) - train_size
train_set, val_set = torch.utils.data.random_split(data_set, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=16,
                                          shuffle=True, num_workers=2) # num_workers: how many subprocesses to use for data loading
val_loader = torch.utils.data.DataLoader(val_set, batch_size=16,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=16,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:04<00:00, 37275334.98it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) # input channel: 3, output channel: 6, kernel size: 5 / (32-5)/1 + 1 = 28 -> 6*28*28
    self.pool = nn.MaxPool2d(2, 2)  # kernel size:2, stride:2 -> 6*14*14
    self.conv2 = nn.Conv2d(6, 16, 5)  # input channel: 6, output channel: 16, kernel size: 5 / (14-5)/1 + 1 = 10 -> 16*10*10
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.out = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)
    return x

In [5]:
model = Model()

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
epochs = 5

for epoch in range(epochs):
  for i, data in enumerate(train_loader, 0):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels.long())
    loss.backward()
    optimizer.step()

    if i % 1000 == 999:
      print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {loss.item()}')

Epoch: 1, Batch: 1000, Loss: 0.9463115334510803
Epoch: 1, Batch: 2000, Loss: 1.2931841611862183
Epoch: 2, Batch: 1000, Loss: 0.9879973530769348
Epoch: 2, Batch: 2000, Loss: 0.7611536979675293
Epoch: 3, Batch: 1000, Loss: 1.0597343444824219
Epoch: 3, Batch: 2000, Loss: 1.3243061304092407
Epoch: 4, Batch: 1000, Loss: 0.7541980147361755
Epoch: 4, Batch: 2000, Loss: 1.021186113357544
Epoch: 5, Batch: 1000, Loss: 1.5352925062179565
Epoch: 5, Batch: 2000, Loss: 0.77165687084198


In [15]:
with torch.no_grad():
    for epoch in range(epochs):
      for i, data in enumerate(val_loader, 0):
        inputs, labels = data

        outputs = model(inputs)
        loss = criterion(outputs, labels.long())
        if i % 200 == 199:
          print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {loss.item()}')

Epoch: 1, Batch: 200, Loss: 1.0397984981536865
Epoch: 1, Batch: 400, Loss: 1.1561123132705688
Epoch: 1, Batch: 600, Loss: 1.0340701341629028
Epoch: 2, Batch: 200, Loss: 1.3677239418029785
Epoch: 2, Batch: 400, Loss: 1.419431447982788
Epoch: 2, Batch: 600, Loss: 1.576132893562317
Epoch: 3, Batch: 200, Loss: 0.9071974754333496
Epoch: 3, Batch: 400, Loss: 0.5714786052703857
Epoch: 3, Batch: 600, Loss: 1.339727520942688
Epoch: 4, Batch: 200, Loss: 0.9577047228813171
Epoch: 4, Batch: 400, Loss: 1.5387508869171143
Epoch: 4, Batch: 600, Loss: 1.450723648071289
Epoch: 5, Batch: 200, Loss: 1.0894684791564941
Epoch: 5, Batch: 400, Loss: 1.2778403759002686
Epoch: 5, Batch: 600, Loss: 0.876646876335144


In [16]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 58 %


In [17]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 66 %
Accuracy of   car : 65 %
Accuracy of  bird : 42 %
Accuracy of   cat : 31 %
Accuracy of  deer : 37 %
Accuracy of   dog : 67 %
Accuracy of  frog : 57 %
Accuracy of horse : 72 %
Accuracy of  ship : 76 %
Accuracy of truck : 66 %
